In [ ]:
from IPython.display import Image
stem_url = "https://raw.githubusercontent.com/James-G-Smith/Python-projects/master/Baker%20Hughes%20Activity%20duration/"

# The Power of Predictive Analytics 



In Hack 6 Baker Hughes asked the community to find out what caused activity duration variance and how to predict it using machine learning. During the Hack Team 9B developed a solution, which was further refined by the Projecting Success. 

The data set includes the following features:
- ID
- Project Number
- Product Line Number
- Department Number
- Activity Type Number
- Code*
- Class Number
- Baseline Start Date
- Baseline Finish Date
- Planned Duration
- Forecast Start Date
- Forecast Finish Date
- Forecast Duration
- Project Duration

*Note that the code feature is hierarchical.

## 1.Import libraries

We import multiple libraries. All are listed below with there uses.

- Pandas - Library for the manipulation of panel data. Think excel.
- Numpy - Library for the manipulation of numerical data.
- Matplotlib - Library for making plots.
- Scikit Learn - Library for Machine Learning (This will be imported later).


In [ ]:
########## IMPORT PACKAGES ##########

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

## 2.Import Data

The data is stored on two excel sheets. These two are imported and merged.

Also, the data is shuffled this is to ensure the data is well mixed. This is needed because the data is in order of activities and if we do not shuffle it there will be unnecessary bias in the data, making the final model worse.   

In [ ]:
# import data
schedule_data = pd.read_csv(stem_url + r"data/P6_Schedule_Data.csv")
project_data = pd.read_csv(stem_url + r"data/Project_Duration.csv")

# merge data
data = pd.merge(schedule_data, project_data, on='ProjectNumber')

# shuffle data
data = data.sample(frac=1).reset_index(drop=True)

## 3.Change Data Type and Clean Data

The data in it's raw form can not be processed by the machine learning model. It needs to be cleaned and converted to the correct data type.

### Converting Data types

In python data can be stored in different types such as string(text data), integers(numerical data), datetime(Date Time Data) and many other data types.

Data features below are converted to a datetime data type. To allow for the model to proccess the data.

In [ ]:
# convert data columns to datetime
data['Baseline Start Date'] = pd.to_datetime(data['Baseline Start Date'])
data['Baseline Finish Date'] = pd.to_datetime(data['Baseline Finish Date'])
data['Forecast Start Date'] = pd.to_datetime(data['Forecast Start Date'])
data['Forecast Finish Date'] = pd.to_datetime(data['Forecast Finish Date'])

Often data can be divided into different into groups. In this case we can divide data on the department number into different groups, as the department number can be 1,2,3,4,5,6,7 or 8. When data can be divided like this we call this **categorical data**.    

There many more examples of categorical data in our data such as:

- ActivityTypeNumber
- ProjectNumber
- ProductLineNumber
- ActivityTypeNumber
- ClassNumber

All our categorical data is current saved as **integers**. This will confuse the model later down the line, so it needs to be converted to a **string**. 


In [ ]:
print(data['ActivityTypeNumber'].dtypes)
print(data['ProjectNumber'].dtypes)
print(data['ProductLineNumber'].dtypes)
print(data['DepartmentNumber'].dtypes)
print(data['ActivityTypeNumber'].dtypes)
print(data['ClassNumber'].dtypes)

Categorical data needs to be converted to **strings**. 

In [ ]:
# convert other columns to strings
data['ActivityTypeNumber'] = data['ActivityTypeNumber'].apply(str)
data['ProjectNumber'] = data['ProjectNumber'].apply(str)
data['ProductLineNumber'] = data['ProductLineNumber'].apply(str)
data['DepartmentNumber'] = data['DepartmentNumber'].apply(str)
data['ActivityTypeNumber'] = data['ActivityTypeNumber'].apply(str)
data['ClassNumber'] = data['ClassNumber'].apply(str)


### Cleaning

Currently, the data includes data on **complete** and **incomplete activities**.

In [ ]:
print(data['ACTIVITY_STATUS'])

We want to look at **completed** activities only.

In [ ]:
completed_data = data[data['ACTIVITY_STATUS'] == 'Completed']
print(completed_data['ACTIVITY_STATUS'])

The activity status needs to be removed, as it provides little insight.  

In [ ]:
completed_data = completed_data.drop(columns = ['ACTIVITY_STATUS'])

## 5.Feature Creation 

One of the best ways to improve the quality of our model is to create **new features** from the **current features**.

### Months 

Currently, the datetime, which includes day, month and year, is saved as one feature. This can be broken into day, month and year. 

To explain how to do this below we extract the month from the start date features and then save them as a new feature.

The day and the year will be extracted in an exercise at the end. 

In [ ]:
# create baseline & forecast start months
completed_data['Baseline Start Month'] = completed_data['Baseline Start Date'].dt.month
completed_data['Forecast Start Month'] = completed_data['Forecast Start Date'].dt.month

### Code

Currently, the code feature is **hierarchical**. This means that data set has a tree structure, where by elements are connect to parent element. 

In [ ]:
filename = "img/Hierachial%20Structure.PNG"

In [ ]:
Image(stem_url + filename)

As you can see below the hierachical structure has not been broken down. 

In [ ]:
print(completed_data['Code'])

The script below splits the code column into each of it's constitute codes. Then the codes are reassambled into new columns.

In [ ]:
# Function to split code column.
def SplitDots(cell):
    return cell.split('.')

completed_data["CodeSplit"] = completed_data["Code"].apply(SplitDots) 

# Now we assemble all the codes.

def Assemble(cell,level):
    elements = len(cell)
    if elements < level:
        return ""
    else:
        code = str(cell[0])
        if level > 1:
            for i in range(level-1):
                code = code + "." + str(cell[i+1])
            return(code)
        else:
            return(code)

completed_data["Code 1"] = completed_data["CodeSplit"].apply(Assemble,level=1)     
completed_data["Code 2"] = completed_data["CodeSplit"].apply(Assemble,level=2)    
completed_data["Code 3"] = completed_data["CodeSplit"].apply(Assemble,level=3)    
completed_data["Code 4"] = completed_data["CodeSplit"].apply(Assemble,level=4)    
completed_data["Code 5"] = completed_data["CodeSplit"].apply(Assemble,level=5)    
completed_data["Code 6"] = completed_data["CodeSplit"].apply(Assemble,level=6)    
completed_data["Code 7"] = completed_data["CodeSplit"].apply(Assemble,level=7)    
completed_data["Code 8"] = completed_data["CodeSplit"].apply(Assemble,level=8)    
completed_data["Code 9"] = completed_data["CodeSplit"].apply(Assemble,level=9)    
completed_data["Code 10"] = completed_data["CodeSplit"].apply(Assemble,level=10)

# remove original code column
completed_data = completed_data.drop(columns = ['Code'])
completed_data = completed_data.drop(columns = ['CodeSplit'])

In [ ]:
print(completed_data['Code 1'])
print('')
print(completed_data['Code 2'])
print('')
print(completed_data['Code 3'])
print('')

### Dummy Variables 

Our data contains a lot of categorical data. Unfortunately, this data can not be processed by our model algorithm. Therefore, we convert our data to dummy variables. A dummy variable is a series of 0s and 1s that can represent a categorgy, which can be processed by the modelling algorithm. 

In [ ]:
# create dummy variables for categorical data
project_dummies = pd.get_dummies(completed_data['ProjectNumber'], prefix='Project ')
product_dummies = pd.get_dummies(completed_data['ProductLineNumber'], prefix='Product Line ')
department_dummies = pd.get_dummies(completed_data['DepartmentNumber'], prefix='Department ')
activity_dummies = pd.get_dummies(completed_data['ActivityTypeNumber'], prefix='Activity Type ')
start_month_dummies = pd.get_dummies(completed_data['Baseline Start Month'], prefix='Month ')
code_1 =  pd.get_dummies(completed_data['Code 1'], prefix='Code 1')
code_2 =  pd.get_dummies(completed_data['Code 2'], prefix='Code 2')
code_3 =  pd.get_dummies(completed_data['Code 3'], prefix='Code 3')
code_4 =  pd.get_dummies(completed_data['Code 4'], prefix='Code 4')
code_5 =  pd.get_dummies(completed_data['Code 5'], prefix='Code 5')
code_6 =  pd.get_dummies(completed_data['Code 6'], prefix='Code 6')
code_7 =  pd.get_dummies(completed_data['Code 7'], prefix='Code 7')
code_8 =  pd.get_dummies(completed_data['Code 8'], prefix='Code 8')
code_9 =  pd.get_dummies(completed_data['Code 9'], prefix='Code 9')
class_dummies = pd.get_dummies(completed_data['ClassNumber'], prefix='Class ')


### Concatenate Data

All the new features need to be concatenated (merged).

In [ ]:
# concatenate final data
final_data = pd.concat([project_dummies, product_dummies, department_dummies, activity_dummies, start_month_dummies, 
                        code_1, code_2, code_3, code_4, code_5, code_6, code_7, code_8, code_9, class_dummies, 
                        completed_data[['Planned Duration','Forecast Duration','ProjectDuration']]], axis=1, sort=False)


## 6.Prepare Data

The data needs to be split into features (X_data) and target(what we are predicting(y_data)). 

In [ ]:
# split into features & target
X_data = final_data.drop(columns = ['Forecast Duration'])
y_data = final_data['Forecast Duration']


### Normalising the Data

Each feature has a **different scale**. This will be a problem as some features will have a greater influence on the model output than others. Therefore, they need to be placed on a common scale, which can be achieved by normalising the data. The StandardScaler function does this.

In [76]:
from sklearn.preprocessing import StandardScaler

# scale the data
scalerX = StandardScaler().fit(X_data)
scalery = StandardScaler().fit(y_data.values.reshape(-1, 1))
X_data = scalerX.transform(X_data)  
y_data = scalery.transform(y_data.values.reshape(-1, 1))  

### Splitting the Data

Before the data can be put into the model it needs to be split into a training and a test data set. 

In splitting the data we create a training set, which is used to create the model and a test set that is used to evaluate the model. 

To split the data the train_test_split function from the scikit-learn library is used. 

In [77]:
from sklearn.model_selection import train_test_split

# split training & validation data
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data,  test_size=0.2)

## 7. Modelling

With the data split, it is now possible to build the model. In this case a **gradient boost regressor** model from scikit-learn is used. 

Inside the **GradientBoostingRegressor function** it is possible to set **hyperparameters**, which are parameters set before the model is created that control the **learning process**. 

Once the hyperparameters have been set, the model can be **fitted** with the **training data**, as seen with **gradient_boosting_model.fit()** function.

Once the **training data** has been fitted to the model, it is time to make a **prediction** based off the **test data**. This is done with **gradient_boosting_model.predict()** function. 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Creation of the model.
gradient_boosting_model = GradientBoostingRegressor(n_estimators=50, max_depth=10)
# Fitting training data on the model.
gradient_boosting_model = gradient_boosting_model.fit(X_train, y_train)
# Predict based off the X_Test
gradient_boosting_predictions = gradient_boosting_model.predict(X_test)

print('done')

C:\Users\james\anaconda3\lib\site-packages\sklearn\ensemble\_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


## 8.Model Evaluation

To evaluate the model R^2 is used.

### **R^2** - Coefficient of Determination

It is a measure how much the model's output variance is influenced by the input variance. 

i.e.  If the R^2 of a model is 0.50, then approximately half of the observed variation can be explained by the model's inputs.

The closer to 0.0 the worse the fit and the closer to 1.0 the better the fit.

The **r2_score** function can be used to calculate **R^2**.

It's important to note scalery.inverse_transform was used to reverse the normalisation implemented before.



In [ ]:
from sklearn.metrics import r2_score

r2_score = r2_score(scalery.inverse_transform(y_test), scalery.inverse_transform(gradient_boosting_predictions))

print('Validation R2 Score: ', r2_score)


## Feature Importance

To understand the influence of each **feature** on the prediction, we calculate the **feature importance**. Feature importance refers to techniques that assign a score to input features based on how useful they are at predicting a target variable.

In this case we only plot the five most infuential features, as the rest have a negligible influence.

In [ ]:
# get feature importance
features = final_data.drop(columns = ['Forecast Duration']).columns.values
importance  = gradient_boosting_model.feature_importances_ 
features_df = pd.DataFrame({'Feature':features, 'Importance':importance})

# plot feature importance
top10_features = features_df.nlargest(5,'Importance')
top10_features.plot.bar(x='Feature')

# Exercises

## 1.  Add up all the values in the planned duration column in the **completed data** set.

The completed_data is a pandas data frame so you'll need to use this code -> df.sum()

## 2. Create a new feature which only includes the date  from the Baseline Start Date from the **completed data** set.

## 3. Create a new feature which only includes the year from the Baseline Start Date from the **completed data** set.

## 4.  Convert the Project Number from the completed data set feature into an integer. 